# Classificador de Imagens - Carro ou Caminhão

A ideia é verificar se é possível classificar de forma simples se determinada imagem é um carro ou caminhão.

Para isso, usaremos o dataset disponível no Kaggle relacionado a isso.

## Importações Iniciais

In [22]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.16.2


In [15]:
caminho_treinamento = "datasets/train"
caminho_validacao = "datasets/valid"

In [18]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

ds_treinamento_ = image_dataset_from_directory(
    caminho_treinamento,
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)

Found 5117 files belonging to 2 classes.


In [19]:
ds_validacao_ = image_dataset_from_directory(
    caminho_validacao,
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)

Found 5051 files belonging to 2 classes.


Lemos as imagens. Agora precisamos juntar todas no mesmo dataset.

In [21]:
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE

ds_treinamento = (
    ds_treinamento_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_validacao = (
    ds_validacao_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

In [25]:
ds_treinamento

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None))>

Criamos nosso modelo de DeepLearning.

In [23]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(128, 128)),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
])

/home/andrevrc/.cache/pypoetry/virtualenvs/car-or-truck-2ldvMJx0-py3.12/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [24]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(ds_treinamento, train_labels, epochs=20, validation_split=0.2)